The first step in kaggle is to get better results in an easy way.

Kaggleの第一歩は楽は方法でより良い結果をえることが重要です。

LightGBM+ optuna　is a very good tool for first trail.Then acheived SCORE 1.08633!!!

LightGBM+ optuna は初めの手にはとても良いツールで、SCORE 1.08633でした。
https://www.kaggle.com/hayahiko/tps-may-easy-way-lgbm-optuna-step-by-step
    
This nootbook is second step exsample.

この方法ノートブックはふたつめの事例です。

Don't forget to vote !!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Reading files from directory
import os
import pickle
 
# Data manipulation & analysis
import pandas as pd
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows', 500)
import datetime as dt
 
import numpy as np
import scipy

# Visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
 
 # 実行に関係ない警告を無視
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import gc
import json
import math

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score,roc_auc_score,log_loss
from sklearn.metrics import mean_squared_error, mean_squared_log_error


from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

from catboost import CatBoostClassifier

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
submission = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")

In [ ]:
display(train.head())
display(train.describe())
display(train.info())

In [ ]:
display(test.head())
display(test.describe())
display(test.info())

In [ ]:
#--------------------------------
#数値の特徴量　※上級者は、特徴量のリストを作成している。
#------------------------------
features_num = ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
       'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
       'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39',
       'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44',
       'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', ]
features_cat = ['target'] 

In [ ]:
# Categorical features distribution 

# Params
n_col = 1
n_row = round(len(features_num) )
size = (n_col * 10, n_row * 4.5)

#Create figure
plt.subplots(n_row,n_col,figsize=size)

# enumerate関数　for - loop コードの代用 
for  i ,feature  in enumerate(features_num , 1):
    plt.subplot(n_row, n_col , i)
    sns.countplot(x = feature, hue = 'target', data = train)
    plt.xlabel(feature, fontsize=9); plt.legend()
plt.show()

In [ ]:
##--------------------------------------------
#カテゴリfeature  　ラベルエンコーダー
#---------------------------------------------
for feature in features_cat :
    le = LabelEncoder()
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])

In [ ]:
# 学習データを特徴量と目的変数に分ける  正解ラべルを記入
train_x = train.drop(['target'], axis=1)
train_y = train['target']

In [ ]:
# 変数Idを除外する
train_x = train_x.drop(['id'], axis=1)
test_x = test.drop(['id'], axis=1)

In [ ]:
train_y

In [ ]:
## Trial 関数で最適化を探索
##計算量を減らすために、Hold-out法で2:8に分割
##パラメータ探索は重要パラメータをsuggest_int(整数で全部探索) , その他をsuggest_categorical（リスト内を選択）

def objective(trial,data=train_x,target=train_y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=71)
    params = {#'iterations':trial.suggest_int("iterations", 4000, 10000),
             'iterations':trial.suggest_int("iterations", 17000, 17000),
             'od_type':'Iter',
              'od_wait':trial.suggest_int('od_wait', 1000, 1000),
              #'od_wait':trial.suggest_int('od_wait', 500, 2300),
             'loss_function':'MultiClass',
              'eval_metric':'MultiClass',
              #'leaf_estimation_method':'Newton',
              #'bootstrap_type': 'Bernoulli',
              #'learning_rate' : trial.suggest_uniform('learning_rate',0.02,0.3),
              'learning_rate' : trial.suggest_uniform('learning_rate',0.01,0.01),
              #'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
              #'subsample': trial.suggest_uniform('subsample',0,1),
              #'random_strength': trial.suggest_uniform('random_strength',10,30),
              #'depth': trial.suggest_int('depth',1,6),
              'depth': trial.suggest_int('depth',4,4), #FIX
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,3),
              'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,3),
              'max_ctr_complexity': trial.suggest_int('max_ctr_complexity',15,15),
               }
    model = CatBoostClassifier(**params)  
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
        
    preds_opt = model.predict_proba(test_x)


    log_loss_multi = log_loss(test_y, preds_opt)
    
    return log_loss_multi

In [ ]:
import optuna
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier
OPTUNA_OPTIMIZATION = True

study = optuna.create_study(direction='minimize')  #'maximize''minimize
study.optimize(objective, n_trials=35)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
if OPTUNA_OPTIMIZATION:
    display(optuna.visualization.plot_optimization_history(study))
    display(optuna.visualization.plot_slice(study))

In [ ]:
################################
#######   Catboost 
################################

train_oof_cat_0 = np.zeros((len(train_x), 4))
temp_test = np.zeros((len(test_x), 4))

cat_params = study.best_trial.params
cat_params['loss_function'] = 'MultiClass'
#cat_params['eval_metric'] = 'MultiClass'
#cat_params['bootstrap_type']= 'Bernoulli'
#cat_params['leaf_estimation_method'] = 'Newton'
cat_params['random_state'] = 42


NUM_FOLDS = 5
kf =  StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=137)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train_x, train_y))):
        print(f'Fold {f+1}')
        train_df = train_x.iloc[train_ind].reset_index(drop=True)
        val_df = train_x.iloc[val_ind].reset_index(drop=True)
        train_target = train_y.iloc[train_ind].reset_index(drop=True)
        val_target = train_y.iloc[val_ind].reset_index(drop=True)

        model =CatBoostClassifier(**cat_params)
        
        model =  model.fit(train_df, train_target,eval_set=[(val_df,val_target)],
                           early_stopping_rounds=30,verbose=False)
        
        temp_oof = model.predict_proba(val_df)
        print(log_loss(val_target, temp_oof))
        train_oof_cat_0[val_ind] = temp_oof

        temp_test += model.predict_proba(test_x)
        test_preds_cat_0 = temp_test/NUM_FOLDS
        test_preds_cat_0 = np.clip(test_preds_cat_0, 0.08, 0.95)
        
print('All_logloss',log_loss(train_y, train_oof_cat_0))
np.save('train_oof_cat_0', train_oof_cat_0 ) #for validation
np.save('test_preds_cat_0',test_preds_cat_0 ) #for submission

In [ ]:
##### 提出用ファイルの作成 ヘッダー無設の設定
submission = pd.DataFrame( test_preds_cat_0)
submission.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4']
submission['id'] = test['id']
submission = submission[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4']]

submission.to_csv("submission_cat.csv", index=False)
display(submission.head(), submission.tail())